In [1]:
import tensorly as tl
from tensorly import decomposition
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})

# Let us use Tucker decomposition to observe unentangled qubits

NB with method only detects single qubits, not their groups!

We will reshape a 4-qubit state into a tensor $\mathbb{R}^{\{0,1\}^4}$.

In [2]:
state = np.array([0, 1, 3, 0.05, 0, 2, 6, 0, 0, 1, 1, 0.05, 0, 2, 2, -0.1])
# normalize the vector to a valid quantum state 
state = state / sum((state ** 2)) ** .5
print(state)
tensor = state.reshape(2, 2, 2, 2)
print(type(tensor), tensor.shape)

[0.0000 0.1291 0.3872 0.0065 0.0000 0.2582 0.7745 0.0000 0.0000 0.1291
 0.1291 0.0065 0.0000 0.2582 0.2582 -0.0129]
<class 'numpy.ndarray'> (2, 2, 2, 2)


This tensor is then resorganized in a Tucker format -- a core tensor, and the matrices, corresponding to each of the modes. As soon as our tensor has the biggest rank of 2 for each mode (qubit), for the rank `(2, 2, 2, 2)` we can reconstruct in back from the Tucker format with no loss.

In [3]:
# the lossless decomposition
t_core, t_matrices = tl.decomposition.tucker(tensor, rank=(2, 2, 2, 2))

In [4]:
t_core.shape

(2, 2, 2, 2)

In [5]:
for m in t_matrices: print(m.shape)

(2, 2)
(2, 2)
(2, 2)
(2, 2)


In [6]:
tensor_restored = tl.tucker_tensor.tucker_to_tensor((t_core, t_matrices))
np.allclose(tensor, tensor_restored)

True

Let us explore some Tucker reduced forms:

In [7]:
for rank in [(1, 2, 2, 2), (2, 1, 2, 2), (2, 2, 1, 2), (2, 2, 2, 1), 
             (1, 1, 2, 2), (2, 2, 1, 1), (2, 1, 2, 1), (1, 2, 1, 2), 
             (2, 1, 1, 2), (1, 2, 2, 1)]:
    t_core, t_matrices = tl.decomposition.tucker(tensor, rank)
    print("Tucker Rank", rank)
    print("\tCore tensor shape", t_core.shape)
    print("\tMatrices shapes  ", *[mx.shape for mx in t_matrices])
    approx_tensor = tl.tucker_tensor.tucker_to_tensor((t_core, t_matrices))
    error = np.linalg.norm((tensor - approx_tensor).flatten())
    print(f"\tReconstruction error {error:.4f}")
    print()

Tucker Rank (1, 2, 2, 2)
	Core tensor shape (1, 2, 2, 2)
	Matrices shapes   (2, 1) (2, 2) (2, 2) (2, 2)
	Reconstruction error 0.1695

Tucker Rank (2, 1, 2, 2)
	Core tensor shape (2, 1, 2, 2)
	Matrices shapes   (2, 2) (2, 1) (2, 2) (2, 2)
	Reconstruction error 0.0129

Tucker Rank (2, 2, 1, 2)
	Core tensor shape (2, 2, 1, 2)
	Matrices shapes   (2, 2) (2, 2) (2, 1) (2, 2)
	Reconstruction error 0.4082

Tucker Rank (2, 2, 2, 1)
	Core tensor shape (2, 2, 2, 1)
	Matrices shapes   (2, 2) (2, 2) (2, 2) (2, 1)
	Reconstruction error 0.4085

Tucker Rank (1, 1, 2, 2)
	Core tensor shape (1, 1, 2, 2)
	Matrices shapes   (2, 1) (2, 1) (2, 2) (2, 2)
	Reconstruction error 0.1698

Tucker Rank (2, 2, 1, 1)
	Core tensor shape (2, 2, 1, 1)
	Matrices shapes   (2, 2) (2, 2) (2, 1) (2, 1)
	Reconstruction error 0.4085

Tucker Rank (2, 1, 2, 1)
	Core tensor shape (2, 1, 2, 1)
	Matrices shapes   (2, 2) (2, 1) (2, 2) (2, 1)
	Reconstruction error 0.4085

Tucker Rank (1, 2, 1, 2)
	Core tensor shape (1, 2, 1, 2)
	Matr

For ranks of 2 modes we observed lower reconstruction error. How can we utilize this for state initialization?

In [8]:
t_core, t_matrices = tl.decomposition.tucker(tensor, (1, 1, 2, 2))

Restore the tensor by hands to check we understand the method correctly.

In [9]:
restored_tensor = t_core

# along the mode 3 we keep the size
restored_tensor = (t_matrices[3] @ restored_tensor.reshape(2, 2).T).T.reshape((1, 1, 2, 2))

# along the mode 2 we keep the size
restored_tensor = (t_matrices[2] @ restored_tensor.reshape(2, t_matrices[2].shape[1])).reshape((1, 1, 2, 2))

# along the mode 1 we make size from 1 to 2
restored_tensor = np.kron(t_matrices[1], restored_tensor.reshape(-1, t_matrices[1].shape[1])).reshape((1, 2, 2, 2))

# along the mode 0 we make size from 1 to 2
restored_tensor = np.kron(t_matrices[0], restored_tensor.reshape(-1, t_matrices[0].shape[1])).reshape((2, 2, 2, 2))
restored_tensor

array([[[[0.0000, 0.1558],
         [0.3762, -0.0003]],

        [[0.0000, 0.3116],
         [0.7524, -0.0005]]],


       [[[0.0000, 0.0645],
         [0.1558, -0.0001]],

        [[0.0000, 0.1291],
         [0.3116, -0.0002]]]])

In [10]:
np.allclose(restored_tensor, tl.tucker_tensor.tucker_to_tensor((t_core, t_matrices)))

True

Let us do the same using tensor products instead of matrix

In [11]:
# along the mode 3 we keep the size
d = (t_matrices[3] @ t_core.reshape(2, 2).T).T.reshape((1, 1, 2, 2))
# along the mode 2 we keep the size
cd = (t_matrices[2] @ d.reshape(2, 2)).reshape((1, 1, 2, 2))

# un-entangled states
a = np.array(t_matrices[0].reshape(2, 1))
b = np.array(t_matrices[1].reshape(2, 1))

final_state = np.kron(np.kron(a, b), cd)
restored_tensor_2 = final_state.reshape((2, 2, 2, 2))
np.allclose(restored_tensor_2, restored_tensor)

True

# Quantum part

Let us put these values to the quantum circuit with initialize algorithm, and see that simulation produces the desired state.

In [16]:
from qiskit import QuantumCircuit, QuantumRegister, execute, BasicAer, quantum_info, transpile

Add precision and convert the normalize states to valid quantum states

In [13]:
# increase precision
a = a.astype(np.float64)
b = b.astype(np.float64)
cd = cd.astype(np.float64)

# norm to quantum states
a = a.reshape(-1) / sum(a.reshape(-1) ** 2) ** .5
b = b.reshape(-1) / sum(b.reshape(-1) ** 2) ** .5
cd = cd.reshape(-1) / sum(cd.reshape(-1) ** 2) ** .5

In [14]:
qr = QuantumRegister(4, "qbits")
qc = QuantumCircuit(qr)

qc.initialize(a, qubits=[3])
qc.initialize(b, qubits=[2])
qc.initialize(cd, qubits=[0, 1])
qc.draw()

┌────────────────────────────────────────────┐
qbits_0: ┤0                                           ├
         │  Initialize(0,-0.38268,-0.92388,0.0006567) │
qbits_1: ┤1                                           ├
         └──────┬─────────────────────────────┬───────┘
qbits_2: ───────┤ Initialize(0.44721,0.89443) ├────────
               ┌┴─────────────────────────────┴┐       
qbits_3: ──────┤ Initialize(-0.92388,-0.38269) ├───────
               └───────────────────────────────┘

In [19]:
backend = BasicAer.get_backend("statevector_simulator")
qc = transpile(qc, backend)
job = execute(qc, backend=backend)
statevector = job.result().get_statevector().real
print(statevector)
print(f"Finally, the fidelity of the prapared state is {quantum_info.state_fidelity(statevector, state):.5f}") 
print(f"CNOT count: {qc.count_ops()['cx']}")

[0.0000 0.1581 0.3817 -0.0003 0.0000 0.3162 0.7634 -0.0005 0.0000 0.0655
 0.1581 -0.0001 0.0000 0.1310 0.3162 -0.0002]
Finally, the fidelity of the prapared state is 0.97116
CNOT count: 2


In [20]:
qr0 = QuantumRegister(4, "qbits")
qc0 = QuantumCircuit(qr)

qc0.initialize(state, qubits=range(4))
qc0.draw()
qc0 = transpile(qc0, backend)
job0 = execute(qc0, backend=backend)
statevector0 = job0.result().get_statevector().real
print(statevector0)
print(f"Finally, the fidelity of the prapared state is {quantum_info.state_fidelity(statevector0, state):.5f}") 
print(f"CNOT count: {qc0.count_ops()['cx']}")

[0.0000 0.1291 0.3872 0.0065 0.0000 0.2582 0.7745 0.0000 0.0000 0.1291
 0.1291 0.0065 0.0000 0.2582 0.2582 -0.0129]
Finally, the fidelity of the prapared state is 1.00000
CNOT count: 22
